In [236]:
!pip install -U selenium

     |████████████████████████████████| 911kB 209kB/s eta 0:00:01


In [280]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime
import pickle
from selenium import webdriver
import pandas as pd

In [151]:
response = requests.get("https://www.goodreads.com/book/show/54226795-mine")

In [152]:
soup = BeautifulSoup(response.text, "lxml")

In [186]:
def find_publisher(publishing_info):
    for info in publishing_info:
        try:
            index = info.index("by")
            return info[index+3:].strip()
        except:
            continue
            

def find_publishing_year(publishing_info):
    years = set()
    for info in publishing_info:
        years = years.union({int(x) for x in re.findall(r"[1-2][0-9]{3}", info)})
        
    return min(years)

def get_book_list(url):
    soup = BeautifulSoup(requests.get(url).text, "lxml")
    book_list = []
    pages_count = int(soup.find("div", class_="pagination").find_all("a")[-2].text)
    for p in range(1, page_count+1):
        soup = BeautifulSoup(requests.get("{}?page={}".format(url, p)).text, "lxml")
        book_list.extend([x["href"] for x in soup.find_all("a", class_="bookTitle", href=True)])
        
    return book_list

            


In [213]:
# classics1 = get_book_list("https://www.goodreads.com/list/show/2522.Cult_Classics")
# popular = get_book_list("https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once")
# ya = get_book_list("https://www.goodreads.com/list/show/43.Best_Young_Adult_Books")
# best_century = get_book_list("https://www.goodreads.com/list/show/6.Best_Books_of_the_20th_Century")
# random1 = get_book_list("https://www.goodreads.com/list/show/1938.What_To_Read_Next")
# random2 = get_book_list("https://www.goodreads.com/list/show/2491.Must_Read_Books_Different_Genres")
# book_clubs = get_book_list("https://www.goodreads.com/list/show/19.Best_for_Book_Clubs")
# non_fiction1 = get_book_list("https://www.goodreads.com/list/show/134.Best_Non_Fiction_no_biographies_")
# non_fiction2 = get_book_list("https://www.goodreads.com/list/show/735.Must_Read_Non_Fiction")
# non_fiction3 = get_book_list("https://www.goodreads.com/list/show/21995.Best_21st_Century_Non_Fiction")

In [196]:
# little_known = get_book_list("https://www.goodreads.com/list/show/25529.Best_Unknown_but_must_be_Known_books_")
# memoirs = get_book_list("https://www.goodreads.com/list/show/281.Best_Memoir_Biography_Autobiography")
# thriller = get_book_list("https://www.goodreads.com/list/show/164293.Thrillers_Must_Reads_")
# depressing = get_book_list("https://www.goodreads.com/list/show/1154.Most_Depressing_Book_of_All_Time")
# science = get_book_list("https://www.goodreads.com/list/show/692.Best_Science_Books_Non_Fiction_Only")
# feminist = get_book_list("https://www.goodreads.com/list/show/62.Best_Feminist_Books")
# philosophical = get_book_list("https://www.goodreads.com/list/show/453.Best_Philosophical_Literature")
# japanese = get_book_list("https://www.goodreads.com/list/show/32.Best_Japanese_Books")
# scifi_fantasy = get_book_list("https://www.goodreads.com/list/show/3.Best_Science_Fiction_Fantasy_Books")
# history = get_book_list("https://www.goodreads.com/list/show/1362.Best_History_Books_")
# historical_fiction = get_book_list("https://www.goodreads.com/list/show/15.Best_Historical_Fiction")

In [201]:
# all_books = set(classics+popular+ya+best_century+random1+random2+book_clubs+non_fiction1+non_fiction2+non_fiction3+little_known+memoirs+thriller+depressing+science+feminist+philosophical+japanese+scifi_fantasy+history+historical_fiction)

In [203]:
# len(all_books)

67384

In [208]:
# # Save into file, as they took forever to get..
# with open('book_urls.txt','wb') as f:
#    pickle.dump(all_books, f)

In [211]:
##To read it again from file
with open('book_urls.txt','rb') as f:
   all_books = pickle.load(f)

In [217]:
all_books = list(all_books)

In [219]:
all_books[:10]

['/book/show/18301603-the-strange-maid',
 '/book/show/2573867-a-darker-domain',
 '/book/show/25111005-a-thousand-naked-strangers',
 '/book/show/13550547-looking-back-with-a-smile',
 '/book/show/20549001-bargain-paradise',
 '/book/show/2399930.Hester_Roon',
 '/book/show/22817757-the-trouble-with-playing-cupid',
 '/book/show/17155071-suicide-watch',
 '/book/show/966098.Paul_Lafargue_and_the_Founding_of_French_Marxism_1842_1882',
 '/book/show/25842727-i-ll-walk-alone']

In [ ]:
title = soup.find("h1", id="bookTitle").text.strip()
authors = [x.find("span", itemprop="name").text for x in soup.find_all("a", class_="authorName")]
description = soup.find("div", id="descriptionContainer").text.strip()
publishing_info = [x.strip() for x in soup.find("div", id="details").find_all(class_="row")[1].text.split("\n") if len(x.strip()) != 0]
publisher = find_publisher(publishing_info)
publishing_year = find_publishing_year(publishing_info)
genres = [tag.text for tag in soup.find_all("a", class_="actionLinkLite bookPageGenreLink")]
page_count = int(re.sub("[^0-9]", "", soup.find("span", itemprop="numberOfPages").text))

In [286]:
driver = webdriver.Safari()
dic = {"Title": [], "Author": [], "Description": [], "Publisher": [], "Year Published": [], "Genres": [], "Page Count": []}

for book in all_books[:10000]:
    driver.get("https://www.goodreads.com"+book)
    soup = BeautifulSoup(driver.page_source, "lxml")
    try:
        dic["Title"].append(soup.find("h1", id="bookTitle").text.strip())
    except: 
        dic["Title"].append(None)
        
    try:
        dic["Author"].append([x.find("span", itemprop="name").text for x in soup.find_all("a", class_="authorName")])
    except:
        dic["Author"].append(None)
        
    try:
        dic["Description"].append(soup.find("div", id="descriptionContainer").text.strip())
    except:
        dic["Description"].append(None)
        
    try:
        publishing_info = [x.strip() for x in soup.find("div", id="details").find_all(class_="row")[1].text.split("\n") if len(x.strip()) != 0]
    except:
        pass
    
    try:
        dic["Publisher"].append(find_publisher(publishing_info))
    except:
        dic["Publisher"].append(None)
        
    try:
        dic["Year Published"].append(find_publishing_year(publishing_info))
    except:
        dic["Year Published"].append(None)
       
    try:
        dic["Genres"].append([tag.text for tag in soup.find_all("a", class_="actionLinkLite bookPageGenreLink")])
    except:
        dic["Genres"].append(None)
        
    try:
        dic["Page Count"].append(int(re.sub("[^0-9]", "", soup.find("span", itemprop="numberOfPages").text)))
    except:
        dic["Page Count"].append(None)



In [287]:
df = pd.DataFrame(dic)

In [288]:
df

,Title,Author,Description,Publisher,Year Published,Genres,Page Count
0,The Strange Maid,[Tessa Gratton],"Fans of Neil Gaiman, Holly Black, and Maggie S...",Random House Books for Young Readers,2014.0,"[Fantasy, Young Adult, Fantasy, Mythology, Fic...",400.0
1,A Darker Domain,[Val McDermid],"1984, Fife. Heiress Catriona Maclennan Grant &...",HarperCollins Publishers,2008.0,"[Mystery, Mystery, Crime, Fiction, Cultural, S...",371.0
2,A Thousand Naked Strangers: A Paramedic's Wild...,[Kevin Hazzard],"A former paramedic’s visceral, poignant, and m...",Scribner,2016.0,"[Nonfiction, Autobiography, Memoir, Medical, B...",261.0
3,Looking Back with a Smile,[Edward B. Farber],"In Looking Back with a Smile, the author recol...",None,2012.0,[],NaN
4,Bargain Paradise,[Darlene Blasing],Bargain Paradise is a humorous tale of an obse...,Argon Press,2013.0,[],49.0
...,...,...,...,...,...,...,...
9995,Cast in Sorrow,[Michelle Sagara],THE END OF HER JOURNEY IS ONLY THE BEGINNING.....,Harlequin Luna,2013.0,"[Fantasy, Fantasy, Magic, Fantasy, Urban Fanta...",480.0
9996,End of the Civil War,[E.E. Doc Murdock],The End of the Civil War is a fast-paced weste...,H.O.T. Press,2013.0,"[Historical, Historical Fiction]",296.0
9997,The Intermediaries: Beat & Case,[Taylor Dye],Angels. Spirits. Superheroes. Gods. Whatever t...,Samanedna Publishers,2014.0,[],338.0
9998,Life Among the Savages,[Shirley Jackson],"Shirley Jackson, author of the classic short s...",Penguin Group,1953.0,"[Nonfiction, Autobiography, Memoir, Humor, Bio...",256.0


In [289]:
df.to_csv("data_10000.csv")

In [276]:
dic

{'Title': ['The Strange Maid',
  'A Darker Domain',
  "A Thousand Naked Strangers: A Paramedic's Wild Ride to the Edge and Back",
  'Looking Back with a Smile',
  'Bargain Paradise',
  'Hester Roon',
  'The Trouble with Playing Cupid',
  'Suicide Watch',
  'Paul Lafargue and the Founding of French Marxism, 1842-1882',
  'I´ll Walk Alone'],
 'Author': [['Tessa Gratton'],
  ['Val McDermid'],
  ['Kevin Hazzard'],
  ['Edward B. Farber'],
  ['Darlene Blasing'],
  ['Norah Lofts'],
  ['Tamara Philip'],
  ['Kelley York'],
  ['Leslie Derfler'],
  ['Mary Higgins Clark']],
 'Description': ['Fans of Neil Gaiman, Holly Black, and Maggie Stiefvater will embrace the richly drawn, Norse-influenced alternate world of the United States of Asgard, where cell phones, rock bands, and evangelical preachers coexist with dragon slaying, rune casting, and sword training in schools. Where the president runs the country alongside a council of Valkyries, gods walk the red car\nFans of Neil Gaiman, Holly Black, an

In [269]:
all_books[:10]

['/book/show/18301603-the-strange-maid',
 '/book/show/2573867-a-darker-domain',
 '/book/show/25111005-a-thousand-naked-strangers',
 '/book/show/13550547-looking-back-with-a-smile',
 '/book/show/20549001-bargain-paradise',
 '/book/show/2399930.Hester_Roon',
 '/book/show/22817757-the-trouble-with-playing-cupid',
 '/book/show/17155071-suicide-watch',
 '/book/show/966098.Paul_Lafargue_and_the_Founding_of_French_Marxism_1842_1882',
 '/book/show/25842727-i-ll-walk-alone']

In [271]:
driver = webdriver.Safari()
driver.get("https://www.goodreads.com/book/show/18301603-the-strange-maid")
soup = BeautifulSoup(driver.page_source, "lxml")
print(soup.find("h1", id="bookTitle").text.strip())
driver.get("https://www.goodreads.com/book/show/2573867-a-darker-domain")
soup = BeautifulSoup(driver.page_source, "lxml")
print(soup.find("h1", id="bookTitle").text.strip())

The Strange Maid
A Darker Domain


'I´ll Walk Alone'

---

### List of books
I cannot use the Goodreads API as keys are not being given anymore.
I will attempt to scrape a big list of books through the Goodreads lists using BeautifulSoup as well.